In [1]:
# default_exp data

In [2]:
from nbdev.showdoc import *

To train, we will use data gratiously shared by authors of [MultiFiT: Efficient Multi-lingual Language Model Fine-tuning](MultiFiT: Efficient Multi-lingual Language Model Fine-tuning). Here is the accompanying [repository](https://github.com/n-waves/multifit).

We are interested in the wikipedia dumps.

In [2]:
!mkdir -p data
!wget -nc 'https://www.dropbox.com/sh/srfwvur6orq0cre/AAAQc36bcD17C1KM1mneXN7fa/data/wiki?dl=1' -O 'data/preprocessed_wiki_8langs.zip'
!unzip 'data/preprocessed_wiki_8langs.zip' -d 'data'

--2020-01-15 15:38:36--  https://www.dropbox.com/sh/srfwvur6orq0cre/AAAQc36bcD17C1KM1mneXN7fa/data/wiki?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.1, 2620:100:6022:1::a27d:4201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/srfwvur6orq0cre/AAAQc36bcD17C1KM1mneXN7fa/data/wiki [following]
--2020-01-15 15:38:36--  https://www.dropbox.com/sh/dl/srfwvur6orq0cre/AAAQc36bcD17C1KM1mneXN7fa/data/wiki
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc2173abafec6a55ac59cf336c1.dl.dropboxusercontent.com/zip_download_get/ATnkaMkel-an1mWzaVuIlopUYOOy0nk5B1YMPr9O1l4CSWePQ1KiiVrW15DtzbgEWZCkMikT41F2FnRHnusM8iMTLKivRPxdPjsLgxv5QcJv9w?dl=1 [following]
--2020-01-15 15:38:37--  https://ucc2173abafec6a55ac59cf336c1.dl.dropboxusercontent.com/zip_download_get/ATnkaMkel-an1mWzaVuIlopUYOOy0nk5B1YMPr9O1l4CSWeP

Let's extract the English and Spanish wiki dumps.

In [11]:
!tar -xvf data/en-100.tar.gz -C data

en-100/
en-100/en.wiki.valid.tokens
en-100/en.wiki.train.tokens
en-100/en.wiki.test.tokens


In [15]:
!tar -xvf data/es-100.tar.gz -C data

es-100/
es-100/es.wiki.test.tokens
es-100/es.wiki.train.tokens
es-100/es.wiki.valid.tokens


In [17]:
!head data/en-100/en.wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomer

In [5]:
!tar -xvf data/es-100.tar.gz

es-100/
es-100/es.wiki.test.tokens
es-100/es.wiki.train.tokens
es-100/es.wiki.valid.tokens


In [9]:
ls data

de-100.tar.gz*  es-100.tar.gz*  ja-100.tar.gz*                README*
en-100.tar.gz*  es-clean.txt    pl-100.tar.gz*                ru-100.tar.gz*
en-clean.txt    fr-100.tar.gz*  pl.txt                        zh-100.tar.gz*
en.txt          it-100.tar.gz*  preprocessed_wiki_8langs.zip


de-100.tar.gz*  es-100.tar.gz*  ja-100.tar.gz*                README*
en-100.tar.gz*  es-clean.txt    pl-100.tar.gz*                ru-100.tar.gz*
en-clean.txt    fr-100.tar.gz*  pl.txt                        zh-100.tar.gz*
en.txt          it-100.tar.gz*  preprocessed_wiki_8langs.zip


We will be working with text. [Project Gutenberg](https://www.gutenberg.org/wiki/Main_Page) is a great source for obtaining books that are in the public domain!

Let's put together some rudimentary mechanism for obtaining data.

In [3]:
#export

import urllib.request
from fastcore.foundation import *
import re

def download_ebook(url, skip_lines=0, drop_lines=None):
    response = urllib.request.urlopen(url)
    data = response.read()
    text = data.decode('utf-8')
    lst =  L(text.split('\r\n'))
    lst = lst.filter(lambda x: x != '').map(lambda x: x.strip()).map(lambda x: re.sub(r'(—|_|“|”)', '', x))
    if drop_lines is not None: drop_lines = -drop_lines
    return ' '.join(lst[skip_lines:drop_lines])

In [4]:
mkdir -p data

In [5]:
pride_and_prejudice_url = 'https://www.gutenberg.org/files/1342/1342-0.txt'
text = download_ebook(pride_and_prejudice_url, 82, 300)

In [6]:
text[:500]

'It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife. However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters. My dear Mr. Bennet, said his lady to him one day, have you heard that Netherfield Park is let at last? Mr. Bennet replied th'

We can now use this functionality to create a couple of small datasets.

In [7]:
en = ''.join(
    [
        download_ebook('https://www.gutenberg.org/files/1342/1342-0.txt', 82, 300), # Pride and Prejudice
        download_ebook('https://www.gutenberg.org/files/4300/4300-0.txt', 100, 340), # Ulysses
        download_ebook('https://www.gutenberg.org/files/16/16-0.txt', 100, 340) # Peter Pan
    ]
)

In [8]:
len(en.split())

431933

In [9]:
with open("data/en.txt", "w") as text_file:
    text_file.write(en)

Unfortunately there are not that many books in Polish on Project Gutenberg, however there is another site, wolnelektury.pl with a lot of books in the public domain.

The focus of this repository will be unsupervised translation and working with English and Polish will be easiest for me.

In [10]:
pl = ''.join(
    [
        download_ebook('https://wolnelektury.pl/media/book/txt/robinson-crusoe.txt', 100, 300), # Robinson Crusoe
        download_ebook('https://wolnelektury.pl/media/book/txt/lord-jim-tom-pierwszy.txt', 100, 300), # Lord Jim
        download_ebook('https://wolnelektury.pl/media/book/txt/kipling-druga-ksiega-dzungli.txt', 100, 300) # Second Jungle Book
    ]
)

In [11]:
len(pl.split())

105637

In [12]:
with open("data/pl.txt", "w") as text_file:
    text_file.write(pl)

In [13]:
from nbdev.export import *
notebook2script()

Converted 00_data.ipynb.
Converted 99_index.ipynb.
Converted language_play_v9.ipynb.
Converted language_play_v9_softmax.ipynb.
